# install

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 16.1 MB/s eta 0:00:00


In [ ]:
!pip install torchsummaryX==1.1.0 -q

In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 48.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyar

In [ ]:
! pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


# lib

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import math
import wandb
import torchsummaryX
import os
from tqdm.notebook import tqdm
import gc
import time

import pandas as pd
from datasets import load_from_disk
import datasets
from datasets import load_dataset
import tiktoken
import torch.nn as nn
from torch.nn import functional as F

import random

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# config

In [ ]:
config = {
    'epochs'        : 4,
    'batch_size'    : 64,
    'init_lr'       : 1e-5,
    "Name"          : 'HW5-pre',
    "d_model"       : 256,
    "n_emb"         : 128,
    "block_size"    :32,
    "n_head"        : 12,
    "n_layer"       : 12,
    "dropout"       : 0.1,
}



device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
enc = tiktoken.get_encoding("gpt2")
vocab_size = enc.n_vocab
vocab_size

50257

# nano - data

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

--2024-04-27 02:08:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-04-27 02:08:51 (28.7 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



# Download Openwebtext Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Skylion007/openwebtext",split=[f'train[{0}%:{25}%]'])

In [ ]:
split_dataset = dataset[0].train_test_split(test_size=0.001, seed=2357, shuffle=True)
split_dataset['val'] = split_dataset.pop('test')

In [ ]:
def process(example):
    ids = enc.encode_ordinary(example['text'])
    ids.append(enc.eot_token)

    out = {'ids': ids, 'len': len(ids)}
    return out

tokenized = split_dataset.map(
    process,
    remove_columns=['text'],
    desc="tokenizing the splits",
)

In [ ]:
for split, dset in tokenized.items():
    arr_len = np.sum(dset['len'], dtype=np.uint64)
    filename = os.path.join(os.path.dirname('/content/'), f'{split}.bin')
    dtype = np.int32
    arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
    total_batches = 512

    idx = 0
    for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
        batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
        arr_batch = np.concatenate(batch['ids'])

        arr[idx : idx + len(arr_batch)] = arr_batch
        idx += len(arr_batch)
    arr.flush()

# get data

In [ ]:
m = np.memmap('/content/train.bin', dtype=np.int32,  mode='r')
arr = np.array(m)
train_data = torch.from_numpy(arr)
#train_data = torch.from_numpy(np.uint16(train_data))

In [ ]:
m = np.memmap('/content/val.bin', dtype=np.int32, mode='r')
arr = np.array(m)
val_data = torch.from_numpy(arr)
#val_data = torch.from_numpy(np.uint16(val_data))

In [ ]:
enc.decode(np.array(train_data[:50]).tolist())

'The police have managed to arrest four members of the EWS scam gang but the mastermind is still on the run.\n\nSome uber rich and elite families in Delhi are fudging income certificates to masquerade as economically weaker sections (EWS) for'

# data loader

In [ ]:
train_data

NameError: name 'train_data' is not defined

In [ ]:
class DataLoaderForLanguageModeling(torch.utils.data.DataLoader): # Inherit from torch.utils.data.DataLoader

    def __init__(self, dataset, batch_size, drop_last= False, sequence_length=128):

        self.dataset    = dataset
        self.batch_size = batch_size
        self.drop_last  = drop_last
        self.sequence_length = sequence_length

        self.num_batches =  len(self.dataset) // (self.batch_size * self.sequence_length)

        if self.drop_last:
            self.dataset = self.dataset[:self.num_batches * self.batch_size * self.sequence_length]
        else:
            required_length = self.num_batches * self.batch_size * self.sequence_length
            padding_length = required_length - len(self.dataset)

            if padding_length > 0:
                self.dataset = np.pad(self.dataset, (0, padding_length), 'constant', constant_values=33279)

    def __len__(self):
        return self.num_batches

    def __iter__(self):

        for i in range(0, len(self.dataset) - self.batch_size * self.sequence_length, self.batch_size * self.sequence_length):

            batch_input_data = self.dataset[i:i + self.batch_size * self.sequence_length]

            batch_target_data = self.dataset[i + 1:i + 1 + self.batch_size * self.sequence_length]

            batch_input_data = batch_input_data.reshape(self.batch_size, self.sequence_length)
            batch_target_data = batch_target_data.reshape(self.batch_size, self.sequence_length)

            yield torch.tensor(batch_input_data), torch.tensor(batch_target_data)

In [ ]:
# Some sanity checks

dl = DataLoaderForLanguageModeling(
    dataset     = np.array(train_data),
    batch_size  = config['batch_size'],
    drop_last   = True,
    # Input Extra parameters here if needed
)

inputs, targets = next(iter(dl))
print(inputs.shape, targets.shape)

In [ ]:
for x, y in dl:
    print("x: ", ''.join([enc.decode(np.array([i]).tolist()) for i in x[0, :]]))
    print("y: ", ''.join([enc.decode(np.array([i]).tolist()) for i in y[0, :]]))
    break

# model

## Positional Embedding

In [ ]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, n_embd, max_len=5000):
        super().__init__()

        pe          = torch.zeros(max_len, n_embd)

        position    = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        div_term    = torch.exp(torch.arange(0, n_embd, 2).float() * (-math.log(10000.0) / n_embd))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):

      return x + self.pe[:, :x.size(1)]

## Attention

In [ ]:
class ScaledDotProductAttention(torch.nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature    = temperature                       # Scaling factor for the dot product
        self.dropout        = torch.nn.Dropout(attn_dropout)    # Dropout layer for attention weights
        self.softmax        = torch.nn.Softmax(dim=2)           # Softmax layer along the attention dimension

    def forward(self, q, k, v, mask=None):

        # Calculate the dot product between queries and keys.
        attn = torch.bmm(q, k.transpose(1, 2))

        # Scale the dot product by the temperature.
        attn = attn / self.temperature

        if mask is not None:
            # Apply the mask by setting masked positions to a large negative value.
            # This ensures they have a softmax score close to zero.
            mask_value = -1e+30 if attn.dtype == torch.float32 else -1e+4
            attn = attn.masked_fill(mask, mask_value)

        # Apply softmax to obtain attention weights.
        attn    = self.softmax(attn)

        # Apply dropout to the attention weights.
        attn    = self.dropout(attn)

        # Compute the weighted sum of values based on the attention weights.
        output  = torch.bmm(attn, v)

        return output, attn # Return the attention output and the attention weights.

In [ ]:
class MultiHeadAttention(torch.nn.Module):
    ''' Multi-Head Attention Module '''

    def __init__(self, n_head, d_model, dropout=0.1):
        super().__init__()

        self.n_head = n_head # Number of attention heads
        self.d_k    = d_model // n_head
        self.d_v    = d_model // n_head


        # Linear layers for projecting the input query, key, and value to multiple heads
        self.w_qs   = torch.nn.Linear(d_model, n_head * self.d_k)
        self.w_ks   = torch.nn.Linear(d_model, n_head * self.d_k)
        self.w_vs   = torch.nn.Linear(d_model, n_head * self.d_v)

        torch.nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + self.d_k)))
        torch.nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + self.d_k)))
        torch.nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + self.d_v)))

        # Initialize the weights of the linear layers
        self.attention = ScaledDotProductAttention(
            temperature=np.power(self.d_k, 0.5), attn_dropout=dropout)

        # Final linear layer to project the concatenated outputs of the attention heads back to the model dimension
        self.fc = torch.nn.Linear(n_head * self.d_v, d_model)

        torch.nn.init.xavier_normal_(self.fc.weight)

        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):

        # following key, value, query standard computation
        d_k, d_v, n_head    = self.d_k, self.d_v, self.n_head
        sz_b, len_q, _      = q.size()
        sz_b, len_k, _      = k.size()
        sz_b, len_v, _      = v.size()

        # Project the input query, key, and value to multiple heads
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        # Rearrange the dimensions to group the heads together for parallel processing
        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv

        # Repeat the mask for each attention head if a mask is provided
        if mask is not None:
            mask = mask.repeat(n_head, 1, 1)

        # Apply scaled dot-product attention to the projected query, key, and value
        output, attn    = self.attention(q, k, v, mask=mask)

        # Rearrange the output back to the original order and concatenate the heads
        output          = output.view(n_head, sz_b, len_q, d_v)
        output          = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output          = self.dropout(self.fc(output))

        return output, attn

## MLP

In [ ]:
class MLP(torch.nn.Module):

    def __init__(self, n_embd, dropout=0.1):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):

        x = self.net(x)

        return x

## Transformer Block

In [ ]:
class transformerBlock(nn.Module):

    def __init__(self, n_embd, n_head, dropout = 0.1):
        super().__init__()
        self.sa = MultiHeadAttention(n_head, n_embd)
        self.mlp = MLP(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.do1   = torch.nn.Dropout(dropout)
        self.do2   = torch.nn.Dropout(dropout)

    def forward(self, x):
        x = x + self.sa(x,x,x)[0]
        n = self.do1(x)
        n = self.ln1(n)
        x = x + self.mlp(n)
        n = self.do2(x)
        n = self.ln2(n)
        return n

## Langugae model (Modified from nanogpt)

In [ ]:
class LanguageModel(nn.Module):

    def __init__(self, n_embd= config['n_emb'], n_head= config['n_head'], n_layer= config['n_layer']):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, n_embd)
        self.pe = PositionalEncoding(n_embd)
        self.blocks = nn.Sequential(*[transformerBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        idx = idx.type(torch.LongTensor).to(device)
        tok_emb = self.embedding(idx)
        x = self.pe(tok_emb)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            targets = targets.type(torch.LongTensor).to(device)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -config['block_size']:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

## model, optimizaer

In [ ]:
epochs = config['epochs']

In [ ]:
model = LanguageModel().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'])

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                T_max = epochs, eta_min=1E-6)

criterion   = torch.nn.CrossEntropyLoss()

scaler = torch.cuda.amp.GradScaler()

In [ ]:
# summary = torchsummaryX.summary(model.to(device), x = torch.tensor(inputs).to(device))

# Trainer

In [ ]:
class Trainer:
    def __init__(self, model, loader, optimizer, criterion, scheduler, scaler, max_epochs= 1, run_id= 'exp'):
        self.model      = model
        self.loader     = loader
        self.optimizer  = optimizer
        self.criterion  = criterion
        self.scheduler  = scheduler
        self.scaler     = scaler

        self.train_losses           = []
        self.val_losses             = []
        self.prediction_probs       = []
        self.prediction_probs_test  = []
        self.generated_texts_test   = []
        self.epochs                 = 0
        self.max_epochs             = max_epochs
        self.run_id                 = run_id


    def calculate_loss(self, out, target):

        out     = out.view(out.shape[0]*out.shape[1], out.shape[2]) # TODO
        targets = target.view(-1)
        loss    = self.criterion(out, targets)

        return loss


    def train(self):

        self.model.train() # set to training mode
        self.model.to(device)
        epoch_loss  = 0
        num_batches = 0
        batch_bar = tqdm(total=len(self.loader), dynamic_ncols=True, leave=False, position=0, desc="Train")

        for batch_num, (inputs, targets) in enumerate(self.loader):

            inputs = torch.tensor(inputs).long().to(device)
            targets = torch.tensor(targets).long().to(device)

            self.optimizer.zero_grad()

            with torch.cuda.amp.autocast():

              outputs,_ = self.model(inputs)
              loss = self.calculate_loss(outputs, targets)

            loss_item = loss.item()
            epoch_loss += loss_item

            scaler.scale(loss).backward()
            scaler.step(self.optimizer)
            scaler.update()

            batch_bar.set_postfix(
                loss = "{:.04f}".format(float(epoch_loss / (batch_num + 1))),
                lr = "{:.04f}".format(float(self.optimizer.param_groups[0]['lr']))
            )
            batch_bar.update()


        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN] \tEpoch [%d/%d] \tLoss: %.4f \tLr: %.6f'
                    % (self.epochs, self.max_epochs, epoch_loss, self.optimizer.param_groups[0]['lr']))
        self.train_losses.append(epoch_loss)

        return (epoch_loss, self.optimizer.param_groups[0]['lr'])


    def save(self): # Don't change this function

        model_path = os.path.join('hw5/experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()}, model_path)


# Pretrain

## wandb

In [ ]:
# using WandB? resume training?

USE_WANDB = True
RESUME_LOGGING = False

# creating your WandB run
run_name = "{}_Transformer_{}_{}".format(
    config["Name"],
    config["d_model"],
    config["n_emb"],)

if USE_WANDB:

    wandb.login(key="ac6c55905bdaf4a86b53f94c3ddf1db807d1a9a1", relogin=True) # TODO enter your key here

    if RESUME_LOGGING:
        run_id = ""
        run = wandb.init(
            id     = run_id,        ### Insert specific run id here if you want to resume a previous run
            resume = True,          ### You need this to resume previous runs, but comment out reinit=True when using this
            project = "HW4P2-S24",  ### Project should be created in your wandb account
        )

    else:
        run = wandb.init(
            name    = run_name,     ### Wandb creates random run names if you skip this field, we recommend you give useful names
            reinit  = True,         ### Allows reinitalizing runs when you re-run this cell
            project = "HW4P2-S24",  ### Project should be created in your wandb account
            config  = config        ### Wandb Config for your run
        )

        ### Save your model architecture as a string with str(model)
        model_arch  = str(model)

        ### Save it in a txt file
        arch_file   = open("model_arch.txt", "w")
        file_write  = arch_file.write(model_arch)
        arch_file.close()

        ### Log it in your wandb run with wandb.save()
        wandb.save("model_arch.txt")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yujiel2. Use `wandb login --relogin` to force relogin


## Experiment

In [ ]:
run_id = str(int(time.time()))
if not os.path.exists('/content/hw5/experiments'):
    os.mkdir('/content/hw5/experiments')
os.mkdir('/content/hw5/experiments/%s' % run_id)
print("Saving models, prediction prbabilities, and generated texts to /content/hw5/experiments/%s" % run_id)

# The object of the Trainer class takes in everything
trainer = Trainer(
    model       = model,
    loader      = dl,

    optimizer   = optimizer,
    criterion   = criterion,
    scheduler   = scheduler,
    scaler      = scaler,
    max_epochs  = config['epochs'],
    run_id      = run_id
)

Saving models, prediction prbabilities, and generated texts to /content/hw5/experiments/1713923049


In [ ]:
wandb.watch(model, log="all")

torch.cuda.empty_cache()
gc.collect()

# %%time
best_nll = 1e30
for epoch in range(config['epochs']):
    train_loss, curr_lr = trainer.train()

    scheduler.step()
    trainer.save()

    wandb.log({"train_loss":train_loss,
               "learning_rate": curr_lr
              })

### Finish your wandb run
run.finish()

Train:   0%|          | 0/275712 [00:00<?, ?it/s]

<ipython-input-20-5a1fb7cc5fe6>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs).long().to(device)
<ipython-input-20-5a1fb7cc5fe6>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets).long().to(device)


Buffered data was truncated after reaching the output size limit.

# Finetune

In [ ]:
model.load_state_dict(torch.load('/content/model-5.pkl')['state_dict'])

<All keys matched successfully>

## summarize

In [ ]:
summary_dataset = load_dataset("cnn_dailymail", '1.0.0')
def process_sum(example):
    ids = enc.encode_ordinary(example['article'])
    ids.extend(enc.encode_ordinary('Here is the summary for the text:') )
    ids.extend(enc.encode_ordinary(example['highlights']))
    ids.append(enc.eot_token)

    out = {'ids': ids, 'len': len(ids)}
    return out
summary_tokenized = summary_dataset.map(
    process_sum,
    desc="tokenizing the splits",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/287113 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/13368 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
summary_tokenized

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'ids', 'len'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'ids', 'len'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'ids', 'len'],
        num_rows: 11490
    })
})

## Question

In [ ]:
question_dataset = load_dataset("rajpurkar/squad")
def process_sum(example):
    ids = enc.encode_ordinary(example['context'])
    ids.extend(enc.encode_ordinary(example['question']))
    ids.extend(enc.encode_ordinary('Here is the answer for the question:') )
    ids.extend(enc.encode_ordinary(example['answers']['text'][0]))
    ids.append(enc.eot_token)

    out = {'ids': ids, 'len': len(ids)}
    return out
question_tokenized = question_dataset.map(
    process_sum,
    desc="tokenizing the splits",
)

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/87599 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
question_tokenized

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'ids', 'len'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'ids', 'len'],
        num_rows: 10570
    })
})

## sentiment

In [ ]:
dataset = load_dataset("financial_phrasebank",'sentences_50agree')

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

In [ ]:
sentiment_dataset = dataset['train'].train_test_split(test_size=0.12, seed=2357, shuffle=True)
sentiment_dataset['val'] = sentiment_dataset.pop('test')

In [ ]:
def process_sum(example):
    ids = enc.encode_ordinary(example['sentence'])
    ids.extend(enc.encode_ordinary('Here is the sentiment label for the sentence:') )
    ids.extend(enc.encode_ordinary(str(example['label'])))
    ids.append(enc.eot_token)

    out = {'ids': ids, 'len': len(ids)}
    return out
sentiment_tokenized = sentiment_dataset.map(
    process_sum,
    desc="tokenizing the splits",
)

tokenizing the splits:   0%|          | 0/4264 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/582 [00:00<?, ? examples/s]

In [ ]:
sentiment_tokenized

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'ids', 'len'],
        num_rows: 4264
    })
    val: Dataset({
        features: ['sentence', 'label', 'ids', 'len'],
        num_rows: 582
    })
})

## NER

In [ ]:
ner_dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
ner_label = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
id2label = {v: k for k, v in ner_label.items()}

In [ ]:
def process_sum(example):
    ids = enc.encode_ordinary(' '.join(example['tokens']))
    ids.extend(enc.encode_ordinary('Here is the name entity recognition label sequence for the sentence:') )
    ids.extend(enc.encode_ordinary(' '.join(str(example['ner_tags']))))
    ids.append(enc.eot_token)

    out = {'ids': ids, 'len': len(ids)}
    return out

ner_tokenized = ner_dataset.map(
    process_sum,
    desc="tokenizing the splits",
)

tokenizing the splits:   0%|          | 0/14041 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/3250 [00:00<?, ? examples/s]

tokenizing the splits:   0%|          | 0/3453 [00:00<?, ? examples/s]

## shaffle

In [ ]:
summary = summary_tokenized.remove_columns(['article', 'highlights', 'id'])
question = question_tokenized.remove_columns(['id', 'title', 'context', 'question', 'answers'])
sentiment = sentiment_tokenized.remove_columns(['sentence', 'label'])
ner = ner_tokenized.remove_columns(['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'])

In [ ]:
train_finetune  = datasets.concatenate_datasets([summary['train'], question['train'], sentiment['train'], ner['train']])
shuffled_train = train_finetune.shuffle(seed=42)

In [ ]:
val_finetune  = datasets.concatenate_datasets([summary['validation'], question['validation'], sentiment['val'], ner['validation']])
shuffled_val = val_finetune.shuffle(seed=42)

In [ ]:
shaffled_data = datasets.DatasetDict({"train":shuffled_train,"test":shuffled_val})

In [ ]:
finetune_data = []
for split, dset in shaffled_data.items():
    arr_len = np.sum(dset['len'], dtype=np.uint64)
    dtype = np.int32
    arr = np.zeros( shape=(arr_len,))
    total_batches = 512

    idx = 0
    for batch_idx in tqdm(range(total_batches)):
        batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
        arr_batch = np.concatenate(batch['ids'])

        arr[idx : idx + len(arr_batch)] = arr_batch
        idx += len(arr_batch)
    finetune_data.append(arr.astype(np.int32))

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

In [ ]:
finetune_data

[array([  464,  5134,   468, ..., 10150,   764, 50256], dtype=int32),
 array([ 3152,  3932,    69, ..., 46173,   764, 50256], dtype=int32)]

## dataloader

In [ ]:
class DataLoaderForLanguageModeling(torch.utils.data.DataLoader): # Inherit from torch.utils.data.DataLoader

    def __init__(self, dataset, batch_size, drop_last= False, sequence_length=128):

        self.dataset    = dataset
        self.batch_size = batch_size
        self.drop_last  = drop_last
        self.sequence_length = sequence_length

        self.num_batches =  len(self.dataset) // (self.batch_size * self.sequence_length)

        if self.drop_last:
            self.dataset = self.dataset[:self.num_batches * self.batch_size * self.sequence_length]
        else:
            required_length = self.num_batches * self.batch_size * self.sequence_length
            padding_length = required_length - len(self.dataset)

            if padding_length > 0:
                self.dataset = np.pad(self.dataset, (0, padding_length), 'constant', constant_values=33279)

    def __len__(self):
        return self.num_batches

    def __iter__(self):

        for i in range(0, len(self.dataset) - self.batch_size * self.sequence_length, self.batch_size * self.sequence_length):

            batch_input_data = self.dataset[i:i + self.batch_size * self.sequence_length]

            batch_target_data = self.dataset[i + 1:i + 1 + self.batch_size * self.sequence_length]

            batch_input_data = batch_input_data.reshape(self.batch_size, self.sequence_length)
            batch_target_data = batch_target_data.reshape(self.batch_size, self.sequence_length)

            yield torch.tensor(batch_input_data), torch.tensor(batch_target_data)

In [ ]:
# Some sanity checks

dl = DataLoaderForLanguageModeling(
    dataset     = np.array(finetune_data[0]),
    batch_size  = config['batch_size'],
    drop_last   = True,
    # Input Extra parameters here if needed
)

inputs, targets = next(iter(dl))
print(inputs.shape, targets.shape)

torch.Size([64, 128]) torch.Size([64, 128])


In [ ]:
for x, y in dl:
    print("x: ", ''.join([enc.decode(np.array([i]).tolist()) for i in x[0, :]]))
    print("y: ", ''.join([enc.decode(np.array([i]).tolist()) for i in y[0, :]]))
    break

x:  The NFL has confirmed that the first two International Series games at Wembley next season will kick off in the afternoon. New York Jets' clash with Miami Dolphins and Buffalo Bills' tie against the Jacksonville Jaguars will both have early kick-off times next October in London. The Jets and Dolphins will kick-off at 2.30pm local time on October 4, while on October 25 the Bills and Jaguars will start at 1.30pm UK time. Two of the three International Series games at Wembley next season will kick off in the afternoon . Detroit Lions' game against Atlanta Falcons last October was the first ever afternoon kick-off at Wembley . October
y:   NFL has confirmed that the first two International Series games at Wembley next season will kick off in the afternoon. New York Jets' clash with Miami Dolphins and Buffalo Bills' tie against the Jacksonville Jaguars will both have early kick-off times next October in London. The Jets and Dolphins will kick-off at 2.30pm local time on October 4, while

## wandb

In [ ]:
# using WandB? resume training?

USE_WANDB = True
RESUME_LOGGING = False

# creating your WandB run
run_name = "{}_Transformer_{}_{}".format(
    config["Name"],
    config["d_model"],
    config["n_emb"],)

if USE_WANDB:

    wandb.login(key="ac6c55905bdaf4a86b53f94c3ddf1db807d1a9a1", relogin=True) # TODO enter your key here

    if RESUME_LOGGING:
        run_id = ""
        run = wandb.init(
            id     = run_id,        ### Insert specific run id here if you want to resume a previous run
            resume = True,          ### You need this to resume previous runs, but comment out reinit=True when using this
            project = "HW5-S24",  ### Project should be created in your wandb account
        )

    else:
        run = wandb.init(
            name    = run_name,     ### Wandb creates random run names if you skip this field, we recommend you give useful names
            reinit  = True,         ### Allows reinitalizing runs when you re-run this cell
            project = "HW5-S24",  ### Project should be created in your wandb account
            config  = config        ### Wandb Config for your run
        )

        ### Save your model architecture as a string with str(model)
        model_arch  = str(model)

        ### Save it in a txt file
        arch_file   = open("model_arch.txt", "w")
        file_write  = arch_file.write(model_arch)
        arch_file.close()

        ### Log it in your wandb run with wandb.save()
        wandb.save("model_arch.txt")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yujiel2. Use `wandb login --relogin` to force relogin


## experiment

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'])

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                T_max = 30, eta_min=1E-8)

In [ ]:
run_id = str(int(time.time()))
if not os.path.exists('/content/hw5/experiments'):
    os.mkdir('/content/hw5/experiments')
os.mkdir('/content/hw5/experiments/%s' % run_id)
print("Saving models, prediction prbabilities, and generated texts to /content/hw5/experiments/%s" % run_id)

# The object of the Trainer class takes in everything
trainer = Trainer(
    model       = model,
    loader      = dl,

    optimizer   = optimizer,
    criterion   = criterion,
    scheduler   = scheduler,
    scaler      = scaler,
    max_epochs  = 30,
    run_id      = run_id
)

Saving models, prediction prbabilities, and generated texts to /content/hw5/experiments/1714185214


In [ ]:
wandb.watch(model, log="all")

torch.cuda.empty_cache()
gc.collect()

# %%time
best_nll = 1e30
for epoch in range(30):
    train_loss, curr_lr = trainer.train()

    scheduler.step()
    trainer.save()

    wandb.log({"train_loss":train_loss,
               "learning_rate": curr_lr
              })

### Finish your wandb run
run.finish()

Train:   0%|          | 0/35054 [00:00<?, ?it/s]

<ipython-input-18-5a1fb7cc5fe6>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs).long().to(device)
<ipython-input-18-5a1fb7cc5fe6>:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets).long().to(device)


Buffered data was truncated after reaching the output size limit.

# test

In [ ]:
enc.encode('I love sunny days.')

[40, 1842, 27737, 1528, 13]

In [ ]:
t = model.generate(idx = torch.tensor([[40, 1842, 27737, 1528, 13]], device=device), max_new_tokens = 100)